In [7]:
import numpy as np
import statsmodels.api as sm
from reba_optim.reba_assess import RebaAssess
from human_moveit_config.human_model import HumanModel
import csv
from scipy.optimize import minimize
from baxter_commander.persistence import dicttostate
import rospy
from numpy.random import uniform
import json

In [2]:
def generate_random_pose(database, A_table, B_table, C_table):
    side = ['right', 'left']
    X = []
    # random pose for A group
    i_n0 = np.random.randint(3)
    i_n1 = np.random.randint(3)
    i_n2 = np.random.randint(3)
    X.append(uniform(database['neck_0']['joints'][i_n0], database['neck_0']['joints'][i_n0 + 1]))
    X.append(uniform(database['neck_1']['joints'][i_n1], database['neck_1']['joints'][i_n1 + 1]))
    X.append(uniform(database['neck_2']['joints'][i_n2], database['neck_2']['joints'][i_n2 + 1]))
    n_val = database['neck_1']['values'][i_n1]
    if database['neck_0']['values'][i_n0] == 1 or database['neck_2']['values'][i_n2] == 1:
        n_val += 1

    i_s0 = np.random.randint(3)
    i_s1 = np.random.randint(5)
    i_s2 = np.random.randint(3)
    X.append(uniform(database['spine_0']['joints'][i_s0], database['spine_0']['joints'][i_s0 + 1]))
    X.append(uniform(database['spine_1']['joints'][i_s1], database['spine_1']['joints'][i_s1 + 1]))
    X.append(uniform(database['spine_2']['joints'][i_s2], database['spine_2']['joints'][i_s2 + 1]))
    s_val = database['spine_1']['values'][i_s1]
    if database['spine_0']['values'][i_s0] == 1 or database['spine_2']['values'][i_s2] == 1:
        s_val += 1
    
    k_vals = []
    for i in range(2):
        i_l0 = np.random.randint(2)
        X.append(uniform(database['knee_0']['joints'][i_l0], database['knee_0']['joints'][i_l0 + 1]))
        k_vals.append(database['knee_0']['values'][i_l0])
    k_val = max(k_vals)
    # get value from table
    A_val = A_table[n_val - 1, s_val - 1, k_val - 1]
    
    # random pose for B group
    sh_vals = []
    for i in range(2):
        i_sh0 = np.random.randint(2)
        i_sh1 = np.random.randint(5)
        X.append(uniform(database[side[i] + '_shoulder_0']['joints'][i_sh0],
                         database[side[i] + '_shoulder_0']['joints'][i_sh0 + 1]))
        X.append(uniform(database[side[i] + '_shoulder_1']['joints'][i_sh1],
                         database[side[i] + '_shoulder_1']['joints'][i_sh1 + 1]))
        sh_vals.append(database[side[i] + '_shoulder_1']['values'][i_sh1])
        if database[side[i] + '_shoulder_0']['values'][i_sh0] == 1:
            sh_vals[-1] += 1
    sh_val = max(sh_vals)

    e_vals = []
    for i in range(2):
        i_e0 = np.random.randint(3)
        X.append(uniform(database[side[i] + '_elbow_0']['joints'][i_e0],
                         database[side[i] + '_elbow_0']['joints'][i_e0 + 1]))
        e_vals.append(database[side[i] + '_elbow_0']['values'][i_e0])
    e_val = max(e_vals)

    w_vals = []
    for i in range(2):
        i_w0 = np.random.randint(3)
        i_w1 = np.random.randint(3)
        i_w2 = np.random.randint(3)
        X.append(uniform(database['wrist_0']['joints'][i_w0], database['wrist_0']['joints'][i_w0 + 1]))
        X.append(uniform(database['wrist_1']['joints'][i_w1], database['wrist_1']['joints'][i_w1 + 1]))
        X.append(uniform(database['wrist_2']['joints'][i_w2], database['wrist_2']['joints'][i_w2 + 1]))
        w_vals.append(database['wrist_1']['values'][i_w1])
        if database['wrist_0']['values'][i_w0] == 1 or database['wrist_2']['values'][i_w2] == 1:
            w_vals[-1] += 1
    w_val = max(w_vals)
    
    # get value from table
    B_val = B_table[e_val - 1, sh_val - 1, w_val - 1]
    
    # get total value from C table
    C_val = C_table[int(A_val - 1), int(B_val - 1)]

    return X, C_val

In [3]:
reba = RebaAssess()
rospy.init_node('find_coefficients')

In [4]:
joint_names = ['neck_0', 'neck_1', 'neck_2', 'spine_0', 'spine_1', 'spine_2',
               'right_knee_0', 'left_knee_0', 'right_shoulder_0', 'right_shoulder_1',
               'left_shoulder_0', 'left_shoulder_1', 'right_elbow_0', 'left_elbow_0',
               'right_wrist_0', 'right_wrist_1', 'right_wrist_2', 'left_wrist_0',
               'left_wrist_1', 'left_wrist_2']

In [5]:
rad_factor = np.pi / 180.
rad_convert = lambda x : x * rad_factor

In [6]:
database = {}
# neck
database['neck_0'] = {}
database['neck_0']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['neck_0']['values'] = [1, 0, 1]
database['neck_1'] = {}
database['neck_1']['joints'] = map(rad_convert, [-45, 10, 20, 45])
database['neck_1']['values'] = [2, 1, 2]
database['neck_2'] = {}
database['neck_2']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['neck_2']['values'] = [1, 0, 1]
# trunk
database['spine_0'] = {}
database['spine_0']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['spine_0']['values'] = [1, 0, 1]
database['spine_1'] = {}
database['spine_1']['joints'] = map(rad_convert, [-20, -10, 10, 20, 60, 90])
database['spine_1']['values'] = [2, 1, 2, 3, 4]
database['spine_2'] = {}
database['spine_2']['joints'] = map(rad_convert, [-45, -15, 15, 45])
database['spine_2']['values'] = [1, 0, 1]
# knee 
database['knee_0'] = {}
database['knee_0']['joints'] = map(rad_convert, [-80, -60, 0])
database['knee_0']['values'] = [2, 1]
# shoulder right
database['right_shoulder_0'] = {}
database['right_shoulder_0']['joints'] = map(rad_convert, [0, 75, 90])
database['right_shoulder_0']['values'] = [1, 0]
database['right_shoulder_1'] = {}
database['right_shoulder_1']['joints'] = map(rad_convert, [-45, -20, 20, 45, 90, 135])
database['right_shoulder_1']['values'] = [2, 1, 2, 3, 4]
# shoulder left
database['left_shoulder_0'] = {}
database['left_shoulder_0']['joints'] = map(rad_convert, [0, 75, 90])
database['left_shoulder_0']['values'] = [1, 0]
database['left_shoulder_1'] = {}
database['left_shoulder_1']['joints'] = map(rad_convert, [45, 20, -20, -45, -90, -135])
database['left_shoulder_1']['values'] = [2, 1, 2, 3, 4]
# elbow right
database['right_elbow_0'] = {}
database['right_elbow_0']['joints'] = map(rad_convert, [0, 60, 100, 145])
database['right_elbow_0']['values'] = [2, 1, 2]
# elbow left
database['left_elbow_0'] = {}
database['left_elbow_0']['joints'] = map(rad_convert, [0, -60, -100, -145])
database['left_elbow_0']['values'] = [2, 1, 2]
# wrist
database['wrist_0'] = {}
database['wrist_0']['joints'] = map(rad_convert, [-45, -10, 10, 45])
database['wrist_0']['values'] = [1, 0, 1]
database['wrist_1'] = {}
database['wrist_1']['joints'] = map(rad_convert, [-30, -15, 15, 30])
database['wrist_1']['values'] = [2, 1, 2]
database['wrist_2'] = {}
database['wrist_2']['joints'] = map(rad_convert, [-30, -10, 10, 30])
database['wrist_2']['values'] = [1, 0, 1]

In [10]:
X = []
Y = []
nb_elem = 5
last_class = 10
nb_by_class = np.zeros(last_class)
nb_total = np.ones(last_class) * nb_elem
while not np.array_equal(nb_by_class, nb_total):
    joints, val = generate_random_pose(database, reba.reba_A_table, reba.reba_B_table, reba.reba_C_table)
    int_val = int(val)
    if nb_by_class[int_val - 1] < nb_elem:
        X.append(joints)
        Y.append(val)
        nb_by_class[int_val - 1] += 1

In [34]:
f = open('/tmp/validation.csv', 'wt')
try:
    writer = csv.writer(f)
    writer.writerow(['Y'] + joint_names)
    for i in range(len(X)):
        writer.writerow([Y[i]] + X[i])
finally:
    f.close()

In [11]:
nb_elem = 100
last_class = 10
state_by_class = []
postures_by_class = []
for i in range(last_class):
    state_by_class.append([])
    postures_by_class.append([])
nb_by_class = np.zeros(last_class)
nb_total = np.ones(last_class) * nb_elem
while not np.array_equal(nb_by_class, nb_total):
    joints, val = generate_random_pose(database, reba.reba_A_table, reba.reba_B_table, reba.reba_C_table)
    val = int(val)
    if nb_by_class[val - 1] < nb_elem:
        posture = {}
        state = {}
        state['name'] = joint_names
        state['position'] = joints
        state_by_class[val - 1].append(dicttostate(state))
        postures_by_class[val - 1].append(state)
        nb_by_class[val - 1] += 1
# save the dict of postures
with open('/tmp/postures.json', 'w') as outfile:
    json.dump(postures_by_class, outfile, sort_keys=True, indent=4)

In [8]:
human = HumanModel(control=True)

In [22]:
class_value = 1
states = state_by_class[class_value - 1]
for s in states:
    human.send_state(s)
    for i in range(len(s.joint_state.name)):
        print s.joint_state.name[i], s.joint_state.position[i]
    print '------------------------------------------------'
    raw_input()

neck_0 -0.277559203583
neck_1 0.210894540075
neck_2 -0.737892922438
spine_0 0.00650989783166
spine_1 -0.0276100366633
spine_2 0.120576862698
right_knee_0 -0.627172141466
left_knee_0 -0.0369446963892
right_shoulder_0 1.3621511408
right_shoulder_1 0.23820894599
left_shoulder_0 1.2427772623
left_shoulder_1 0.284672734013
right_elbow_0 1.34810104866
left_elbow_0 -1.32838236501
right_wrist_0 0.505041774613
right_wrist_1 -0.374135913291
right_wrist_2 0.146039718703
left_wrist_0 0.295631630178
left_wrist_1 -0.294640844691
left_wrist_2 0.289561672918
------------------------------------------------

neck_0 0.394365704046
neck_1 0.201919975151
neck_2 0.448563294147
spine_0 0.0316187841871
spine_1 -0.0671261000094
spine_2 0.144138039337
right_knee_0 -0.231997229257
left_knee_0 -0.668417706744
right_shoulder_0 1.40825756331
right_shoulder_1 0.345788340768
left_shoulder_0 1.46116150524
left_shoulder_1 -0.716326763646
right_elbow_0 0.874093440086
left_elbow_0 -2.21309334931
right_wrist_0 -0.0063337